# Multi-Point Proximity encoding
This notebook illustrates the general idea of MPP encoding.
It does not actually use the package, but instead does the encoding 
directly using `shapely` and `numpy` operations.

In [ ]:
import numpy as np
import pandas as pd
import shapely
import shapely.wkt
from shapely.ops import nearest_points

import plotly
from plotly.subplots import make_subplots
from plotly.graph_objects import Scatter

In [ ]:
# Define a region.
x0, y0 = 0, 0
x1, y1 = 400, 300

# Spacing of sample points.
scale = 100

# Set X and Y coords of all reference points.
x_coords = np.arange(x0 + scale / 2, x1, scale)
y_coords = np.arange(y0 + scale / 2, y1, scale)
gg = np.meshgrid(x_coords, y_coords)
rp_x = gg[0].flatten()
rp_y = gg[1].flatten()

# Get a list of geometry objects for the reference points
rp_geom = [
    shapely.wkt.loads('POINT(%f %f)' % a)
    for a in list(zip(rp_x, rp_y))
]

## Plot three geometry types, with encodings.

In [ ]:
geometries = [
    shapely.wkt.loads('POINT(287 122)'),
    shapely.wkt.loads('LINESTRING(40 70, 160 110, 213 190, 285 205)'),
    shapely.wkt.loads('POLYGON((70 20, 210 105, 127 240, 70 20))')
]

# geometries = [
#     shapely.wkt.loads('MULTIPOINT(287 122, 113 207)'),
#     shapely.wkt.loads('MULTILINESTRING((40 70, 160 110, 213 190, 285 205), (220 140, 305 110, 345 30))'),
#     shapely.wkt.loads('MULTIPOLYGON(((70 20, 210 105, 127 240, 70 20)), ((260 225, 270 170, 330 95, 335 185, 260 225)))')
# ]

In [ ]:
fig = plotly.subplots.make_subplots(
    rows=3, cols=2,
    specs=[
        [{"type": "scatter"}, {"type": "table"}],
        [{"type": "scatter"}, {"type": "table"}],
        [{"type": "scatter"}, {"type": "table"}],
    ],
    print_grid=False
)

irow = 0

for geom in geometries:

    irow += 1
    gt = geom.geom_type

    # Get the MPP encodings. Not much to it!
    distances = np.round(np.array([geom.distance(z) for z in rp_geom]), 1)
    factors = np.round(np.exp(-1.0 * distances / scale), 4)    

    # Connection lines.
    xx = []
    yy = []
    for rp in rp_geom:
        p = nearest_points(geom, rp)[0]
        connector_x = [p.xy[0][0], rp.xy[0][0]]
        connector_y = [p.xy[1][0], rp.xy[1][0]]
        xx = xx + connector_x + [None]
        yy = yy + connector_y + [None]
    trace = Scatter(
        x=xx, y=yy, name='distances',
        mode='lines', line={'dash': 'dash', 'color': 'gray'}
    )
    fig.append_trace(trace, irow, 1)

    # Draw the shape.
    if gt == 'Polygon':
        coords = geom.exterior.coords
        xx = [z[0] for z in coords]
        yy = [z[1] for z in coords]
        trace = Scatter(
            x=xx, y=yy, name=gt, 
            mode='lines', marker={'color': 'red'}, fill='toself'
        )

    elif gt == 'MultiPolygon':
        for g in geom.geoms:
            coords = g.exterior.coords
            xx = [z[0] for z in coords]
            yy = [z[1] for z in coords]
            trace = Scatter(
                x=xx, y=yy, name=gt, 
                mode='lines', marker={'color': 'red'}, fill='toself'
            )
            fig.append_trace(trace, irow, 1)
    
    elif gt == 'Point':
        xx = [geom.xy[0][0]]
        yy = [geom.xy[1][0]]
        trace = Scatter(
            x=xx, y=yy, name=gt, 
            mode='markers', marker={'color': 'red', 'size': 12}
        )
        
    elif gt == 'MultiPoint':
        xx = [z.xy[0][0] for z in geom.geoms]
        yy = [z.xy[1][0] for z in geom.geoms]
        trace = Scatter(
            x=xx, y=yy, name=gt, 
            mode='markers', marker={'color': 'red', 'size': 12}
        )
        
    elif gt == 'LineString':
        coords = geom.coords
        xx = [z[0] for z in coords]
        yy = [z[1] for z in coords]
        trace = Scatter(
            x=xx, y=yy, name=gt, 
            mode='lines', marker={'color': 'red', 'size': 12}
        )
    elif gt == 'MultiLineString':
        xx = []
        yy = []
        for g in geom.geoms:
            coords = g.coords
            xx += [z[0] for z in coords] + [None]
            yy += [z[1] for z in coords] + [None]
        trace = Scatter(
            x=xx, y=yy, name=gt, 
            mode='lines', marker={'color': 'red', 'size': 12}
        )

    if gt != 'MultiPolygon':
        fig.append_trace(trace, irow, 1)

    # Reference points.
    trace = Scatter(
        x=rp_x, y=rp_y, name='reference points', 
        mode='markers', marker={'color': 'blue', 'size': 10}
    )
    fig.append_trace(trace, irow, 1)

    fig['layout']['xaxis%d' % irow]['range'] = [x0, x1]
    fig['layout']['yaxis%d' % irow]['range'] = [y0, y1]

    # Encoding summary
    rp_strings = ['(%d, %d)' % z for z in list(zip(rp_x, rp_y))]
    trace = plotly.graph_objects.Table(
        header = {'values': ['ref (x,y)', 'distance', 'encoding']},
        cells = {'values': [rp_strings, distances, factors]}
    )
    fig.append_trace(trace, irow, 2)

fig['layout']['width'] = 1000
fig['layout']['height'] = 1200
fig


## Polygons with holes
Show how this works for polygons with holes.

In [ ]:
# geom = shapely.wkt.loads('POLYGON((80 190, 260 230, 370 150, 310 30, 120 25, 80 190))')
geom = shapely.wkt.loads("""
POLYGON((80 190, 260 230, 370 150, 310 30, 120 25, 80 190), 
(290 150, 220 85, 130 110, 110 170, 240 205, 290 150)
)
""")
distances = np.array([geom.distance(z) for z in rp_geom])
factors = np.exp(-1.0 * distances / scale)

In [ ]:
fig = plotly.subplots.make_subplots(
    rows=2, cols=2,
    specs=[
        [{"type": "scatter"}, {"type": "table"}],
        [{"type": "scatter"}, {"type": "table"}],
    ],
    print_grid=False
)

##
## Filled polygon
##
irow = 1
geom = shapely.wkt.loads('POLYGON((80 190, 260 230, 370 150, 310 30, 120 25, 80 190))')
distances = np.round(np.array([geom.distance(z) for z in rp_geom]), 1)
factors = np.round(np.exp(-1.0 * distances / scale), 4)

# Plot nearest point indicators.
xx = []
yy = []
for rp in rp_geom:
    p = nearest_points(geom, rp)[0]
    connector_x = [p.xy[0][0], rp.xy[0][0]]
    connector_y = [p.xy[1][0], rp.xy[1][0]]
    xx = xx + connector_x + [None]
    yy = yy + connector_y + [None]
trace = Scatter(
    x=xx, y=yy, name='distances',
    mode='lines', line={'dash': 'dash', 'color': 'gray'}
)
fig.append_trace(trace, irow, 1)

# Polygon: Interiors:
for g in geom.interiors:
    inner_x = [z[0] for z in g.coords]
    inner_y = [z[1] for z in g.coords]
    trace = Scatter(
        x=inner_x, y=inner_y, name='hole', fill='none',
        marker={'color': 'red'}
    )
    fig.append_trace(trace, irow, 1)

coords = geom.exterior.coords
outer_x = [z[0] for z in coords]
outer_y = [z[1] for z in coords]
trace = Scatter(
    x=outer_x, y=outer_y, name='polygon', fill='tonext',
    marker={'color': 'red'}
)
fig.append_trace(trace, irow, 1)


# Plot reference points
trace = Scatter(
    x=rp_x, y=rp_y, name='reference points', 
    mode='markers', marker={'color': 'blue', 'size': 10}
)
fig.append_trace(trace, irow, 1)

# Encoding summary
trace = plotly.graph_objects.Table(
    header = {'values': ['ref x', 'ref y', 'distance', 'encoding']},
    cells = {'values': [rp_x, rp_y, distances, factors]}
)
fig.append_trace(trace, irow, 2)

##
## Holy polygon
##
irow = 2
geom = shapely.wkt.loads("""
POLYGON((80 190, 260 230, 370 150, 310 30, 120 25, 80 190), 
(290 150, 220 85, 130 110, 110 170, 240 205, 290 150)
)
""")
distances = np.round(np.array([geom.distance(z) for z in rp_geom]), 1)
factors = np.round(np.exp(-1.0 * distances / scale), 4)

# Plot nearest point indicators.
xx = []
yy = []
for rp in rp_geom:
    p = nearest_points(geom, rp)[0]
    connector_x = [p.xy[0][0], rp.xy[0][0]]
    connector_y = [p.xy[1][0], rp.xy[1][0]]
    xx = xx + connector_x + [None]
    yy = yy + connector_y + [None]
trace = Scatter(
    x=xx, y=yy, name='distances',
    mode='lines', line={'dash': 'dash', 'color': 'gray'}
)
fig.append_trace(trace, irow, 1)

# Polygon: Interiors:
for g in geom.interiors:
    inner_x = [z[0] for z in g.coords]
    inner_y = [z[1] for z in g.coords]
    trace = Scatter(
        x=inner_x, y=inner_y, name='hole', fill='none',
        marker={'color': 'red'}
    )
    fig.append_trace(trace, irow, 1)

coords = geom.exterior.coords
outer_x = [z[0] for z in coords]
outer_y = [z[1] for z in coords]
trace = Scatter(
    x=outer_x, y=outer_y, name='polygon', fill='tonext',
    marker={'color': 'red'}
)
fig.append_trace(trace, irow, 1)

# Plot reference points
trace = Scatter(
    x=rp_x, y=rp_y, name='reference points', 
    mode='markers', marker={'color': 'blue', 'size': 10}
)
fig.append_trace(trace, irow, 1)

# Encoding summary
trace = plotly.graph_objects.Table(
    header = {'values': ['ref x', 'ref y', 'distance', 'encoding']},
    cells = {'values': [rp_x, rp_y, distances, factors]}
)
fig.append_trace(trace, irow, 2)

fig['layout']['width'] = 900
fig['layout']['height'] = 850
fig
